In [32]:
# Dependencies
import pandas as pd
import json
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float, Date
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

In [33]:
# read in happiness files for all years
happiness_file_15 = "Data/happiness_2015.csv"
happy15_df = pd.read_csv(happiness_file_15)

happiness_file_16 = "Data/happiness_2016.csv"
happy16_df = pd.read_csv(happiness_file_16)

happiness_file_17 = "Data/happiness_2017.csv"
happy17_df = pd.read_csv(happiness_file_17)

# read in region and continents data
region_cont = "Data/region_continent.csv"
region_cont_df = pd.read_csv(region_cont)


In [37]:
# Create YEAR and UNIQUE_ID fields
# 2015
happy15_df['YEAR'] = 2015
happy15_df['UNIQUE_ID'] = happy15_df['COUNTRY'] + '_' + happy15_df.YEAR.map(str)

# 2016
happy16_df['YEAR'] = 2016
happy16_df['UNIQUE_ID'] = happy16_df['COUNTRY'] + '_' + happy16_df.YEAR.map(str)

# 2017
happy17_df['YEAR'] = 2017
happy17_df['UNIQUE_ID'] = happy17_df['COUNTRY'] + '_' + happy17_df.YEAR.map(str)

In [43]:
#Merge to capture Region and Continent data
happy15_region_df = pd.merge(happy15_df, region_cont_df, on="COUNTRY")
happy16_region_df = pd.merge(happy16_df, region_cont_df, on="COUNTRY")
happy17_region_df = pd.merge(happy17_df, region_cont_df, on="COUNTRY")

In [46]:
# Set index
happy15_region_df.set_index("UNIQUE_ID", inplace=True)
happy16_region_df.set_index("UNIQUE_ID", inplace=True)
happy17_region_df.set_index("UNIQUE_ID", inplace=True)

In [48]:
# Combine all year DF's into one DF
happyall_region_df = pd.concat([happy15_region_df, happy16_region_df, happy17_region_df])

,COUNTRY,HAPPINESS_RANK,HAPPINESS_SCORE,ECONOMY_GDP_PER_CAPITA,FAMILY,HEALTH_LIFE_EXPECTANCY,FREEDOM,GENEROSITY,TRUST_GOVERNMENT_CORRUPTION,DYSTOPIA_RESIDUAL,YEAR,REGION,CONTINENT
UNIQUE_ID,,,,,,,,,,,,,
Switzerland_2015,Switzerland,1,7.587,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978,2.51738,2015,Europe,Europe
Iceland_2015,Iceland,2,7.561,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145,2.70201,2015,Europe,Europe
Denmark_2015,Denmark,3,7.527,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357,2.49204,2015,Europe,Europe
Norway_2015,Norway,4,7.522,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503,2.46531,2015,Europe,Europe
Canada_2015,Canada,5,7.427,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957,2.45176,2015,North America,North America


In [66]:
#Reorder fields for final output
all_cols = ['YEAR','COUNTRY','REGION','CONTINENT','HAPPINESS_RANK','HAPPINESS_SCORE','ECONOMY_GDP_PER_CAPITA',
                      'FAMILY','HEALTH_LIFE_EXPECTANCY','FREEDOM','GENEROSITY','TRUST_GOVERNMENT_CORRUPTION','DYSTOPIA_RESIDUAL']
happyall_region_df2= happyall_region_df[all_cols].copy()

In [68]:
#Create database connection
##Note: {your_password} should be inserted
connection_string = "root:{your_password}$@localhost/happiness_data"
engine = create_engine(f'mysql://{connection_string}')

# confirm tables
engine.table_names()

['happiness_by_region_yr']

In [71]:
# load final df into sql database
happyall_region_df2.to_sql(name='happiness_by_region_yr', con=engine, if_exists='append', index=True)